# Domain Background
The Google landmark Recognition Challenge on Kaggle is selected to be a capstone project [1]. This topic is interested to me not only that the image recognition is a fundamental problem in computer vision of machine learning but also that this type of question is always faced in the real world.
It is a good topic to a machine-learning beginner because we have learned similar problem in Udacity classes. However, how to apply what we learned into a real case is a totally different scenario. If we can try to solve this type of problem, it will give us an experience of interacting and dealing with a real world case during the problem solving process. On the other hand, landmark recognition is useful to remind users what it is or introduce it to travelers who pass by. For example, travelers took a picture with a building, but forgot the name of this building when they try to put some information on social media or share to their friends. By using this recognition tool, it is easy to remind travelers what the building is. Moreover, when people see a landmark, they would like to know more information of it. This machine-learning model can recognize this image and provides a specifiec information to people.
# Problem Statement
The landmark recognition is a typical problem of computer vision. However, it is a little different from the regular problem. In this case, there are 15k classes of landmarks. The traditional image recognition on ImageNet is able to recognize 1k classes. That means the difficulty is how to recognize this large number of classes. The basic idea is building a CNN model and trains this model, but it causes much computation time to reach an acceptable accuracy. The potential solution is using a better architecture or transferring learning to reduce the computation time and get higher accuracy.
# Evaluation Metrics
The evaluation metrics is Global Average Precession (GAP) at k, where k=1 [3]. For each image, the model has to predict its landmark with confidence score. If there are N predictions for one image, the predictions should be sorted by their confidence score and apply to a function as follows.  
$$GAP = \frac{1}{M}\sum_{i=1}^N P(i) rel(i)$$
where:
* N is the total number of predictions returned by the system, across all
queries
* M is the total number of queries with at least one landmark from the training
set visible in it (note that some queries may not depict landmarks)
* P(i) is the precision at rank i
* Rel(i) denotes the relevance of prediction i: it’s 1 if the i-th prediction is
correct and 0 otherwise


# Data Exploration
Download train data and test data

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

ImportError: No module named 'seaborn'